In [5]:
# https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils import data
from torchvision import transforms
import importlib.util
spec = importlib.util.spec_from_file_location("module.name", "/home/arnab/Desktop/dnn_offloading/Models/bdddataloader.py")
bddloader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(bddloader)


def assign_weight_bias_ff(classifier,model,input_units,prev_input_units):
    num_classes = 12
    for key,value in classifier.state_dict().items():
        k = "classifier." + str(key)
        x = key.split(".")
        if x[1] == "weight":
            if x[0] == "0": # "1" to update weight size of first FF layer based on first layer input
                weight = model[k]
                weight = weight[:,prev_input_units: (prev_input_units + input_units)]
                classifier[int(x[0])].weight.data = weight.cpu()
            elif x[0] == "6": # last linear layer weight row to class_num
                weight = model[k]
                weight = weight[:num_classes,:]
                classifier[int(x[0])].weight.data = weight.cpu()
            else:
                classifier[int(x[0])].weight.data = model[k].cpu()
        elif x[1] == "bias":
            if x[0] == "6": # last linear layer bias to class_num
                bias = model[k]
                bias = bias[:num_classes]
                classifier[int(x[0])].bias.data = bias.cpu()
            else:
                classifier[int(x[0])].bias.data = model[k].cpu()
        
    return classifier
  
in_channel = np.zeros((2),dtype=int) # worker = 2
in_channel[0] = 3
in_channel[1] = 3
prev_input_units = 0
def node(NN,img,partition_inx,current_layer,BATCH_SIZE,channel,model,fog_node):
    global prev_input_units,in_channel
    out_channel = np.zeros((2),dtype=int) # worker = 2
    weight = None
    bias = None
    cfgs = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
    model_list = create_model_list(model)
    model_dict = create_model_dict(cfgs,model_list)
    if NN == "conv":
        img_part = adaptive_partitioning(img,partition_inx)
        if len(model_dict[current_layer]) > 1:
            cfg = model_dict[current_layer][0]
            out_channel[fog_node] = model_dict[current_layer][0]
            weight = model[model_dict[current_layer][1]]
            bias = model[model_dict[current_layer][2]]
        else:
            cfg = model_dict[current_layer]
            
        if cfg == 'M':
            print("\tBefore MaxPool: {}".format(img_part.size()))
            m = nn.MaxPool2d(kernel_size=2, stride=2)
            print("\t" + str(m))
            return m(img_part)
        else:
            print("\tBefore Conv: {}".format(img_part.size()))
            m0 = nn.Conv2d(in_channel[fog_node], out_channel[fog_node], kernel_size=3, padding=1)
            m1 = nn.ReLU(inplace=True)
            print("\t" + str(m0))
            m0.weight.data = weight
            m0.bias.data = bias
            in_channel[fog_node] = out_channel[fog_node]
            return m1(m0(img_part))
    elif NN == "ff":
        num_classes = 12
        img_part = adaptive_partitioning(img,partition_inx)
        img_part = torch.flatten(img_part,1)
        input_units = img_part.shape[1]
        print("\tBefore ff: {}".format(img_part.size()))
        classifier = nn.Sequential(
            nn.Linear(input_units, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        print("\t" + str(classifier))
        classifier = assign_weight_bias_ff(classifier,model,input_units,prev_input_units)        
        prev_input_units = input_units
        
        return classifier(img_part)
        
    #print(m.state_dict().keys())
    
def adaptive_partitioning(img,partition_size):
    index = partition_size
    temp = img.detach().numpy()
    temp = torch.from_numpy(temp[:,:,index[0]:index[1],:])
    return temp

def partition(NN,img,k,CA,f):
    # intializing variables
    A = None
    partition_size = []
    batch = img.shape[0]
    channel = img.shape[1]
    W = [0] * k  # partition
    Windex = []  # partitioning indexes
    init = [0] * (k + 1)
    CA[0] = 0
    CA = [float(val) for val in CA]

    r = img.shape[2]
    c = img.shape[3]
    
    if r > c:
        m = r
    else:
        m = c
    
    
    # sum of capabilities of all nodes
    C2 = np.sum(CA[:(k + 1)])
    
    for i in range(1,k+1):
        C1 = np.sum(CA[:i+1])
        Pi = C1 / C2
        if NN == "conv":
            init[i] = math.floor(Pi * (m - (f - 1)))
            partition_size.append((init[i-1],init[i]+(f-1)))
        elif NN == "ff":
            init[i] = math.floor(Pi * m)
            partition_size.append((init[i - 1],init[i]))

    return partition_size


def create_model_list(model):
    model_list = []
    for key in model:
        model_list.append(key)
        
    return model_list

def create_model_dict(cfg,model_list):
    model_dict = {}
    model_inx = 0
    for i,v in enumerate(cfg):
        if v != 'M':
            model_dict.update({i:(v,model_list[model_inx],model_list[model_inx+1])})
            model_inx += 2
        else:
            model_dict.update({i:v})
    return model_dict

def main():
    # Configuration of network
    cfgs = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
    
    model = torch.load("/home/arnab/Desktop/Data/vgg16.pth", map_location=torch.device('cpu'))
    # Number of workers
    worker = 2
    
    # Capabilities of nodes
    #CA = np.random.randint(1, 10, size=k+1)
    CA = [0.0, 4.0, 6.0]
    print("Fog CA: {}".format(CA))
    
    # Initialize variables
    kernel_filters = 3
    num_conv_layer = 5
    current_layer = None
    partition_list = {}
    classify_list = []
    after_part = None
    input_ = None
    channel = None
    final_out = None
    BATCH_SIZE = 1
    IMAGE_DIM = 224
    
    # load images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    
    loader = data.DataLoader(
        bddloader.BDDDataset('/home/arnab/Desktop/Data', train=True, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    
    print("Image Size: torch.Size([BATCH_SIZE, CHANNEL, ROW, COL])\n")
    inx = 0
    for img,level in loader:
        print(img.size())
        print("=> Image : {}".format(inx+1))

        # Convolutional NN
        for i in range(len(cfgs)):
            current_layer = i
            if i == 0:
                input_ = img
                channel = input_.shape[1]
            else:
                input_ = final_out
                channel = input_.shape[1]

            key = "Conv" + str(i)
            print("=> Convolution Layers: {}".format(i))
            after_part = partition("conv",input_,worker,CA,kernel_filters)
            partition_list.update({key : after_part})

            # processing and marging
            f_inx = None
            s_inx = None
            final_out = None
            for j in range(len(after_part)):
                out = node("conv",input_,after_part[j],current_layer,BATCH_SIZE,channel,model,j)
                print("\tAfter: {}\n".format(out.size()))
                if j == 0:
                    final_out = out
                else:
                    final_out = torch.cat((final_out,out),2)
                    
            print("\tAfter Marge: " + str(final_out.size()))

        
        # Adaptive average Pool
        avgpool = nn.AdaptiveAvgPool2d((7, 7))
        out = avgpool(final_out)
        print(out.size())
        
        # Fully Connected NN
        current_layer += 1 ## remove later
        input_ = out
        channel = input_.shape[1]
        key = "ff"
        print("=> Fully Connected Layers: ")
        after_part = partition("ff",input_,worker,CA,f=0)
        partition_list.update({key : after_part})
        for j in range(len(after_part)):
            out = node("ff",input_,after_part[j],current_layer,BATCH_SIZE,channel,model,j)
            print("\tAfter ff: {}\n".format(out.size()))
            m = nn.ReLU()
            out = m(out).data > 0
            out = out.int()
            classify_list.append(out)
            print("{}\n".format(out))

        
        classify_final = None
        for i in range(len(classify_list)-1):
            if i == 0:
                classify_final = np.bitwise_or(classify_list[i].numpy()[:], classify_list[i+1].numpy()[:]) 
            else:
                classify_final = np.bitwise_or(classify_final,classify_list[i+1].numpy()[:])

        print("Final Feature Classification: {}".format(torch.Tensor(classify_final).double()))
        
        inx += 1
        break # one image break
    
if __name__ == '__main__':
    main()
    

Fog CA: [0.0, 4.0, 6.0]
Image Size: torch.Size([BATCH_SIZE, CHANNEL, ROW, COL])

torch.Size([1, 3, 224, 224])
=> Image : 1
=> Convolution Layers: 0
	Before Conv: torch.Size([1, 3, 90, 224])
	Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
	After: torch.Size([1, 64, 90, 224])

	Before Conv: torch.Size([1, 3, 136, 224])
	Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
	After: torch.Size([1, 64, 136, 224])

	After Marge: torch.Size([1, 64, 226, 224])
=> Convolution Layers: 1
	Before Conv: torch.Size([1, 64, 91, 224])
	Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
	After: torch.Size([1, 64, 91, 224])

	Before Conv: torch.Size([1, 64, 137, 224])
	Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
	After: torch.Size([1, 64, 137, 224])

	After Marge: torch.Size([1, 64, 228, 224])
=> Convolution Layers: 2
	Before MaxPool: torch.Size([1, 64, 92, 224])
	MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
